In [4]:
# %load ../../scripts/lib/utils.py
import psutil
from os import system
from subprocess import Popen,PIPE
from time import time

import numpy as np
from numpy import sqrt, mean
#utils that do not require plotting
from subprocess import Popen,PIPE
from os import system
from os.path import isfile,getmtime

import cv2

#import matplotlib.pyplot as plt
#from astropy.convolution import Gaussian2DKernel,convolve

import yaml

class configuration():
    def __init__(self,yamlFile):
        self.D=yaml.load(open(yamlFile,'r'))

    def getParams(self):
        return self.D

def run(command):
    print('cmd=',command)
    system(command)
    
def runPipe(command):
    print('runPipe cmd=',command)
    p=Popen(command.split(),stdout=PIPE,stderr=PIPE)
    L=p.communicate()
    stdout=L[0].decode("utf-8").split('\n')
    stderr=L[1].decode("utf-8").split('\n')
    return stdout,stderr

time_log=[]
def clock(message):
    print('%8.1f \t%s'%(time(),message))
    time_log.append((time(),message))

def printClock():
    t=time_log[0][0]
    for i in range(1,len(time_log)):
        print('%8.1f \t%s'%(time_log[i][0]-t,time_log[i][1]))
        t=time_log[i][0]

def list_s3_files(stack_directory):
    stdout,stderr=runPipe("aws s3 ls %s/ "%(stack_directory))
    filenames=[]
    for line in stdout:
        parts=line.strip().split()
        if len(parts)!=4:
            continue
        filenames.append(parts[-1])
    return filenames

def read_files(s3_dir,_delete=False,data_dir='/dev/shm/data/'):
    s3files=list_s3_files(s3_dir)
    for filename in s3files:
        if not isfile(data_dir+'/'+filename):
            run('aws s3 cp %s/%s %s'%(s3_dir,filename,data_dir))
        D=fromfile(data_dir+'/'+filename,dtype=np.float16)
        pics=D.reshape([-1,_size,_size])
        if _delete:
            run('rm %s/%s'%(data_dir,filename))
        yield pics

def data_stream(s3_dir='s3://mousebraindata-open/MD657/permuted'):
    for pics in read_files(s3_dir):
        j=0
        for i in range(pics.shape[0]):
            if j%1000==0:
                print('\r examples read=%10d'%j,end='')
            j+=1    
            yield pics[i,:,:]


def Last_Modified(file_name):
    try:
        mtime = getmtime(file_name)
    except OSError:
        mtime = 0
    return(mtime)
            
# def calc_err(pic,gaussian = None):
#     if gaussian is None:
#         gaussian=Gaussian2DKernel(1,x_size=7,y_size=7)
#     factor=np.sum(gaussian)
#     P=convolve(pic,gaussian)/factor
#     #except:
#     #    print('err in calc_err/convolve',pic.shape,gaussian.shape,factor)
#     #    P=pic
#     error=sqrt(mean(abs(pic-P)))
#     sub=P[::2,::2]
#     return error,sub

# def plot_patches(fig,data,h=15,w=15,_titles=[]):
#     for i in range(h*w):
#         if i>=data.shape[0]:
#             break
#         ax=fig.add_subplot(h,w,i+1);
#         pic=np.array(data[i,:,:],dtype=np.float32)

#         subfig=ax.imshow(pic,cmap='gray')
#         if(len(_titles)>i):
#             plt.title(_titles[i])
#         subfig.axes.get_xaxis().set_visible(False)
#         subfig.axes.get_yaxis().set_visible(False)

def pack_pics(Reps):
    size=Reps[0].shape[0]
    _len=len(Reps)
    Reps_mat=np.zeros([_len,size,size])
    for i in range(_len):
        Reps_mat[i,:,:]=Reps[i]
    return Reps_mat

def dist2(a,b):
    diff=(a-b)**2
    return np.sum(diff.flatten())

# def dist_hist(data):
#     D=[]
#     for i in range(1,data.shape[0]):
#         D.append(dist2(data[i,:,:],data[i-1,:,:]))
#         if i%1000==0:
#             print('\r',i,end='')
#     hist(D,bins=100);

# def find_threshold(image,percentile):
#     """find the threshold at the given percentile

#     :param image: grey-level image
#     :param percentile: threshold is chosen so that 
#                        percentile of the pixels are lower than it
#     :returns: threshold
#     :rtype: float

#     """
#     V=sorted(image.flatten())
#     l=len(V)
#     thr=V[int(l*percentile)] 
#     return thr

def mark_contours(D,tile):

    image = np.array(tile,dtype=np.uint8)
    kernel = np.ones((3,3),np.uint8)
    boundary=np.zeros(image.shape,np.uint8)
    repress=boundary.copy()
    #print('shape of boundary=',boundary.shape)
    _shape=D[0]['mask'].shape
    left=int(_shape[0]/2)
    right=_shape[0]-left
    left,right

    for R in D:
        #compute contour
        color=np.array([0,0,0],dtype=np.uint8)
        color[R['j'] % 2]=255
        mask=np.array(R['mask']*1,dtype=np.uint8)
        dilated = cv2.dilate(mask,kernel,iterations = 1)
        contour = dilated-mask
        #mark contour in ln image coordinates
        coor=[R['Y'],R['X']]

        boundary[coor[0]-left:coor[0]+right,coor[1]-left:coor[1]+right]\
        +=np.multiply.outer(contour,color)
        repress[coor[0]-left:coor[0]+right,coor[1]-left:coor[1]+right]\
        +=np.multiply.outer(contour,np.array([1,1,1],dtype=np.uint8))

    combined=image.copy()
    combined[repress==1]=255
    # combined +=boundary

    return combined


In [43]:
find_and_lock('s3://mousebraindata-open/MD657')

runPipe cmd= aws s3 ls s3://mousebraindata-open/MD657/ 
VQ4.pkl no match
VQ5.pkl no match
tiles_612_extracted.pkl no match
cmd= aws s3 cp /tmp/MD657-F1-2017.02.17-17.39.26_MD657_3_0003.lock-yoav-freunds-PowerBook-G4-retina-2+local s3://mousebraindata-open/MD657/MD657-F1-2017.02.17-17.39.26_MD657_3_0003.lock-yoav-freunds-PowerBook-G4-retina-2+local


'MD657-F1-2017.02.17-17.39.26_MD657_3_0003'

In [46]:
!aws s3 rm 's3://mousebraindata-open/MD657/*.lock*'


delete: s3://mousebraindata-open/MD657/*.lock*


In [50]:
_list=!aws s3 ls s3://mousebraindata-open/MD657/

In [51]:
_list

'                           PRE /'

In [44]:
list_s3_files('s3://mousebraindata-open/MD657')

runPipe cmd= aws s3 ls s3://mousebraindata-open/MD657/ 


['MD657-F1-2017.02.17-17.39.26_MD657_1_0001.lock-yoav-freunds-PowerBook-G4-retina-2+local',
 'MD657-F1-2017.02.17-17.39.26_MD657_1_0001.png',
 'MD657-F1-2017.02.17-17.39.26_MD657_1_0001.tif',
 'MD657-F1-2017.02.17-17.39.26_MD657_1_0001_lossless.jp2',
 'MD657-F1-2017.02.17-17.39.26_MD657_1_0001_lossless_patches.tgz',
 'MD657-F1-2017.02.17-17.39.26_MD657_1_0001_lossy.jp2',
 'MD657-F1-2017.02.17-17.39.26_MD657_2_0002.lock-yoav-freunds-PowerBook-G4-retina-2+local',
 'MD657-F1-2017.02.17-17.39.26_MD657_2_0002.png',
 'MD657-F1-2017.02.17-17.39.26_MD657_2_0002.tif',
 'MD657-F1-2017.02.17-17.39.26_MD657_2_0002_lossless.jp2',
 'MD657-F1-2017.02.17-17.39.26_MD657_2_0002_lossless_patches.tgz',
 'MD657-F1-2017.02.17-17.39.26_MD657_2_0002_lossy.jp2',
 'MD657-F1-2017.02.17-17.39.26_MD657_3_0003.lock-yoav-freunds-PowerBook-G4-retina-2+local',
 'MD657-F1-2017.02.17-17.39.26_MD657_3_0003.png',
 'MD657-F1-2017.02.17-17.39.26_MD657_3_0003.tif',
 'MD657-F1-2017.02.17-17.39.26_MD657_3_0003_lossless.jp2',
 

In [28]:
get_file_table('s3://mousebraindata-open/MD657')

runPipe cmd= aws s3 ls s3://mousebraindata-open/MD657/ 
matched, groups= ('MD657-F1-2017.02.17-17.39.26_MD657_1_0001', '.png')
matched, groups= ('MD657-F1-2017.02.17-17.39.26_MD657_1_0001', '.tif')
matched, groups= ('MD657-F1-2017.02.17-17.39.26_MD657_1_0001', '_lossless.jp2')
matched, groups= ('MD657-F1-2017.02.17-17.39.26_MD657_1_0001', '_lossless_patches.tgz')
matched, groups= ('MD657-F1-2017.02.17-17.39.26_MD657_1_0001', '_lossy.jp2')
matched, groups= ('MD657-F1-2017.02.17-17.39.26_MD657_2_0002', '.png')
matched, groups= ('MD657-F1-2017.02.17-17.39.26_MD657_2_0002', '.tif')
matched, groups= ('MD657-F1-2017.02.17-17.39.26_MD657_2_0002', '_lossless.jp2')
matched, groups= ('MD657-F1-2017.02.17-17.39.26_MD657_2_0002', '_lossless_patches.tgz')
matched, groups= ('MD657-F1-2017.02.17-17.39.26_MD657_2_0002', '_lossy.jp2')
matched, groups= ('MD657-F1-2017.02.17-17.39.26_MD657_3_0003', '.png')
matched, groups= ('MD657-F1-2017.02.17-17.39.26_MD657_3_0003', '.tif')
matched, groups= ('MD657-F1-

matched, groups= ('MD657-N1-2017.02.27-15.52.27_MD657_1_0001', '.png')
matched, groups= ('MD657-N1-2017.02.27-15.52.27_MD657_1_0001', '.tif')
matched, groups= ('MD657-N1-2017.02.27-15.52.27_MD657_1_0001', '_lossless.jp2')
matched, groups= ('MD657-N1-2017.02.27-15.52.27_MD657_1_0001', '_lossless_patches.tgz')
matched, groups= ('MD657-N1-2017.02.27-15.52.27_MD657_1_0001', '_lossy.jp2')
matched, groups= ('MD657-N1-2017.02.27-15.52.27_MD657_2_0002', '.png')
matched, groups= ('MD657-N1-2017.02.27-15.52.27_MD657_2_0002', '.tif')
matched, groups= ('MD657-N1-2017.02.27-15.52.27_MD657_2_0002', '_lossless.jp2')
matched, groups= ('MD657-N1-2017.02.27-15.52.27_MD657_2_0002', '_lossless_patches.tgz')
matched, groups= ('MD657-N1-2017.02.27-15.52.27_MD657_2_0002', '_lossy.jp2')
matched, groups= ('MD657-N1-2017.02.27-15.52.27_MD657_3_0003', '.png')
matched, groups= ('MD657-N1-2017.02.27-15.52.27_MD657_3_0003', '.tif')
matched, groups= ('MD657-N1-2017.02.27-15.52.27_MD657_3_0003', '_lossless.jp2')
matc

matched, groups= ('MD657-N75-2017.02.22-18.48.18_MD657_2_0224', '_lossy.jp2')
matched, groups= ('MD657-N76-2017.02.22-18.51.41_MD657_1_0226', '.png')
matched, groups= ('MD657-N76-2017.02.22-18.51.41_MD657_1_0226', '.tif')
matched, groups= ('MD657-N76-2017.02.22-18.51.41_MD657_1_0226', '_lossless.jp2')
matched, groups= ('MD657-N76-2017.02.22-18.51.41_MD657_1_0226', '_lossless_patches.tgz')
matched, groups= ('MD657-N76-2017.02.22-18.51.41_MD657_1_0226', '_lossy.jp2')
matched, groups= ('MD657-N76-2017.02.22-18.51.41_MD657_2_0227', '.png')
matched, groups= ('MD657-N76-2017.02.22-18.51.41_MD657_2_0227', '.tif')
matched, groups= ('MD657-N76-2017.02.22-18.51.41_MD657_2_0227', '_lossless.jp2')
matched, groups= ('MD657-N76-2017.02.22-18.51.41_MD657_2_0227', '_lossless_patches.tgz')
matched, groups= ('MD657-N76-2017.02.22-18.51.41_MD657_2_0227', '_lossy.jp2')
matched, groups= ('MD657-N77-2017.02.22-18.55.01_MD657_1_0229', '.png')
matched, groups= ('MD657-N77-2017.02.22-18.55.01_MD657_1_0229', '.

{'MD657-F1-2017.02.17-17.39.26_MD657_1_0001': ['.png',
  '.tif',
  '_lossless.jp2',
  '_lossless_patches.tgz',
  '_lossy.jp2'],
 'MD657-F1-2017.02.17-17.39.26_MD657_2_0002': ['.png',
  '.tif',
  '_lossless.jp2',
  '_lossless_patches.tgz',
  '_lossy.jp2'],
 'MD657-F1-2017.02.17-17.39.26_MD657_3_0003': ['.png',
  '.tif',
  '_lossless.jp2',
  '_lossless_patches.tgz',
  '_lossy.jp2'],
 'MD657-F10-2017.02.17-19.35.42_MD657_1_0028': ['.png',
  '.tif',
  '_lossless.jp2',
  '_lossless_patches.tgz',
  '_lossy.jp2'],
 'MD657-F10-2017.02.17-19.35.42_MD657_3_0030': ['.png',
  '.tif',
  '_lossless.jp2',
  '_lossless_patches.tgz',
  '_lossy.jp2'],
 'MD657-F10-2017.02.21-17.57.42_MD657_1_0028': ['.png',
  '.tif',
  '_lossless.jp2',
  '_lossless_patches.tgz',
  '_lossy.jp2'],
 'MD657-F10-2017.02.21-17.57.42_MD657_2_0029': ['.png',
  '.tif',
  '_lossless.jp2',
  '_lossless_patches.tgz',
  '_lossy.jp2'],
 'MD657-F10-2017.02.21-17.57.42_MD657_3_0030': ['.png',
  '.tif',
  '_lossless.jp2',
  '_lossless_pa

In [69]:
%%writefile Controller.py
#!/usr/bin/env python3
from os import mkdir
import argparse
from lib.utils import *
import re
import socket
"""
The controller runs a given script on a set of files on S3. The application is intended to run on a set of ec2 instances in parallel. 
Lock files are used to insure that each file is processed exactly once.

This python file will be rewritten in a simpler way using datajoin
"""
def get_file_table(s3_directory,pattern=r'(.*_\d_\d{4})(.*)'):
    """generate a dictionary of the files in an s3 directory 
    that fit a given regex pattern.

    :param s3_directory: s3 directory, example: s3://mousebraindata-open/MD657/
    :param pattern: a regular expression defining the files to be considered.
    :returns: A dictionary in which the file name stem is the key and the value is a list of descriptors of files which have that stem.
    :rtype: 

    """
    
    stdout = list_s3_files(s3_directory)
    pat=re.compile(pattern)

    T={}
    for filename in stdout:
        m=pat.match(filename)
        if m:
            #print('matched, groups=',m.groups())
            file,ext= m.groups()
            if file in T:
                T[file].append(ext)
            else:
                T[file]=[ext]
        else:
             print(filename,'no match')
    return T

def find_and_lock(s3_directory):
    """ find an s3 file without a lock and lock it

    :param s3_directory: location of files and locks
    :param pattern: regular expression that should match the file stem
    """
    T=get_file_table(s3_directory)

    found=False
    for filename,extensions in T.items():
        if '_lossless.jp2' in extensions:
            found=True
            for ext in extensions:
                if '.lock' in ext:
                    found=False
                    break
        if found:
            break
    if not found:
        return None
        
    #create a lock
    hostname=socket.gethostname().replace('.','+')
    flagname=filename+'.lock-'+hostname
    open('/tmp/'+flagname,'w').write(flagname+'\n')

    command='aws s3 cp %s %s/%s'%('/tmp/'+flagname,s3_directory,flagname)
    run(command)

    return filename

if __name__=="__main__":
    
    parser = argparse.ArgumentParser()
    parser.add_argument("scripts_dir", type=str,
                        help="path to the directory with the scripts")
    parser.add_argument("script",type=str,
                        help='the name of the script that is to run on each file')
    parser.add_argument("s3location", type=str,
                        help="path to the s3 directory with the lossless images")
    parser.add_argument("local_data",type=str,
                        help="path to the local data directory")
    # pattern=r'(.*)\.([^\.]*)$'
    args = parser.parse_args()
    scripts_dir=args.scripts_dir
    script=args.script
    s3_directory=args.s3location
    local_data=args.local_data

    time_log=[]
    clock('starting Controller with s3_directory=%s, local_data=%s'%(s3_directory,local_data))


    #preparations: make dirs data and data/tiles
    try:
        #run('sudo chmod 0777 /dev/shm/')
        mkdir(local_data)
        mkdir(local_data+'/tiles')
        clock('created data directory')
    except:
        pass

    #main loop
    while True:
        #find an unprocessed file on S3
        stem=find_and_lock(s3_directory)
        clock('found and locked %s'%stem)

        if stem==None:
            print('all files processed')
            break

        cmd='python3 %s/%s %s %s %s'\
            %(scripts_dir,script,s3_directory,stem+'_lossless',local_data)
        run(cmd)
        
    printClock()

Overwriting Controller.py


In [70]:
%cd ../../scripts/


[Errno 2] No such file or directory: '../../scripts/'
/Users/yoavfreund/projects/shapeology/shapeology_code/scripts


In [71]:
!./Controller.py -h

usage: Controller.py [-h] scripts_dir script s3location local_data

positional arguments:
  scripts_dir  path to the directory with the scripts
  script       the name of the script that is to run on each file
  s3location   path to the s3 directory with the lossless images
  local_data   path to the local data directory

optional arguments:
  -h, --help   show this help message and exit


In [73]:
!./Controller.py . process_file.py s3://mousebraindata-open/MD657 ../../data

1545439436.5 	starting Controller with s3_directory=s3://mousebraindata-open/MD657, local_data=../../data
runPipe cmd= aws s3 ls s3://mousebraindata-open/MD657/ 
VQ4.pkl no match
VQ5.pkl no match
tiles_612_extracted.pkl no match
cmd= aws s3 cp /tmp/MD657-F12-2017.02.17-19.55.34_MD657_1_0034.lock-yoav-freunds-PowerBook-G4-retina-2+local s3://mousebraindata-open/MD657/MD657-F12-2017.02.17-19.55.34_MD657_1_0034.lock-yoav-freunds-PowerBook-G4-retina-2+local
upload: ../../../../../../tmp/MD657-F12-2017.02.17-19.55.34_MD657_1_0034.lock-yoav-freunds-PowerBook-G4-retina-2+local to s3://mousebraindata-open/MD657/MD657-F12-2017.02.17-19.55.34_MD657_1_0034.lock-yoav-freunds-PowerBook-G4-retina-2+local
1545439441.8 	found and locked MD657-F12-2017.02.17-19.55.34_MD657_1_0034
cmd= python3 ./process_file.py s3://mousebraindata-open/MD657 MD657-F12-2017.02.17-19.55.34_MD657_1_0034_lossless ../../data
1545439442.0 	starting to process s3://mousebraindata-open/MD657/MD657-F12-2017.02.17-19.55.34_MD657_

    37                            load:  15.12

load low enough 15.124999999999998
got lock ../../data/tiles/tiles_229.lock 38
cmd= python3 run_job.py ../../data/tiles/tiles_229
    38                            load:  14.44

load low enough 14.4375
got lock ../../data/tiles/tiles_228.lock 39
cmd= python3 run_job.py ../../data/tiles/tiles_228
    39                            load:  15.21

load low enough 15.212499999999999
got lock ../../data/tiles/tiles_214.lock 40
cmd= python3 run_job.py ../../data/tiles/tiles_214
    40                            load:  15.41

load low enough 15.412499999999998
got lock ../../data/tiles/tiles_200.lock 41
cmd= python3 run_job.py ../../data/tiles/tiles_200
    41                            load:  15.95

load low enough 15.95
got lock ../../data/tiles/tiles_162.lock 42
cmd= python3 run_job.py ../../data/tiles/tiles_162
    42                            load:  15.22

load low enough 15.225
got lock ../../data/tiles/tiles_176.lock 43
cmd= python3 run_jo

    86                            load:  15.70

load low enough 15.700000000000001
got lock ../../data/tiles/tiles_58.lock 87
cmd= python3 run_job.py ../../data/tiles/tiles_58
    87                            load:  17.25

load low enough 17.25
got lock ../../data/tiles/tiles_114.lock 88
cmd= python3 run_job.py ../../data/tiles/tiles_114
    88                            load:  16.89

load low enough 16.887500000000003
got lock ../../data/tiles/tiles_100.lock 89
cmd= python3 run_job.py ../../data/tiles/tiles_100
    89                            load:  15.51

load low enough 15.512500000000001
got lock ../../data/tiles/tiles_128.lock 90
cmd= python3 run_job.py ../../data/tiles/tiles_128
    90                            load:  16.64

load low enough 16.6375
got lock ../../data/tiles/tiles_129.lock 91
cmd= python3 run_job.py ../../data/tiles/tiles_129
    91                            load:  15.81

load low enough 15.8125
got lock ../../data/tiles/tiles_101.lock 92
cmd= python3 run_job

   135                            load:  18.02

load low enough 18.025
got lock ../../data/tiles/tiles_51.lock 136
cmd= python3 run_job.py ../../data/tiles/tiles_51
   136                            load:  18.43

load low enough 18.425
got lock ../../data/tiles/tiles_79.lock 137
cmd= python3 run_job.py ../../data/tiles/tiles_79
   137                            load:  24.56

load low enough 24.5625
got lock ../../data/tiles/tiles_135.lock 138
cmd= python3 run_job.py ../../data/tiles/tiles_135
   138                            load:  25.94

load low enough 25.9375
got lock ../../data/tiles/tiles_86.lock 139
cmd= python3 run_job.py ../../data/tiles/tiles_86
   139                            load:  40.81

load low enough 40.8125
got lock ../../data/tiles/tiles_92.lock 140
cmd= python3 run_job.py ../../data/tiles/tiles_92
   140                            load:  18.45

load low enough 18.45
got lock ../../data/tiles/tiles_121.lock 141
cmd= python3 run_job.py ../../data/tiles/tiles_121
   1

   184                            load:  34.31

load low enough 34.3125
got lock ../../data/tiles/tiles_153.lock 185
cmd= python3 run_job.py ../../data/tiles/tiles_153
   185                            load:  18.27

load low enough 18.275
got lock ../../data/tiles/tiles_147.lock 186
cmd= python3 run_job.py ../../data/tiles/tiles_147
   186                            load:  17.85

load low enough 17.85
got lock ../../data/tiles/tiles_219.lock 187
cmd= python3 run_job.py ../../data/tiles/tiles_219
   187                            load:  19.70

load low enough 19.700000000000003
got lock ../../data/tiles/tiles_225.lock 188
cmd= python3 run_job.py ../../data/tiles/tiles_225
   188                            load:  15.66

load low enough 15.662500000000001
got lock ../../data/tiles/tiles_230.lock 189
cmd= python3 run_job.py ../../data/tiles/tiles_230
   189                            load:  15.10

load low enough 15.1
got lock ../../data/tiles/tiles_224.lock 190
cmd= python3 run_job.py ../

1545439950.2 	created tar file ../../data/MD657-F12-2017.02.17-19.55.34_MD657_1_0034_lossless_patches.tgz
cmd= aws s3 cp ../../data/MD657-F12-2017.02.17-19.55.34_MD657_1_0034_lossless_patches.tgz s3://mousebraindata-open/MD657/
upload: ../../data/MD657-F12-2017.02.17-19.55.34_MD657_1_0034_lossless_patches.tgz to s3://mousebraindata-open/MD657/MD657-F12-2017.02.17-19.55.34_MD657_1_0034_lossless_patches.tgz
1545439950.8 	copy tar file to S3
1545439950.8 	finished processing s3://mousebraindata-open/MD657/MD657-F12-2017.02.17-19.55.34_MD657_1_0034_lossless
     0.0 	cleaning local directory
    57.0 	copied from s3: MD657-F12-2017.02.17-19.55.34_MD657_1_0034_lossless
    60.2 	translated into tif
   123.4 	broke into tiles
   267.6 	1 - processed    231 tiles
     0.0 	2 - processed      0 tiles
     0.0 	created tar file ../../data/MD657-F12-2017.02.17-19.55.34_MD657_1_0034_lossless_patches.tgz
     0.7 	copy tar file to S3
     0.0 	finished processing s3://mousebraindata-open/MD657/MD6

In [56]:
!python ./process_file.py -h

usage: process_file.py [-h] s3location stem local_data

A script that takes in an S3 directory breaks it into tiles and extracts
normalized patches from these tiles.

positional arguments:
  s3location  path to the s3 directory with the lossless images
  stem        the file name stem
  local_data  path to the local data directory

optional arguments:
  -h, --help  show this help message and exit
